<a href="https://colab.research.google.com/github/QorbQuant/Crypto-tax-data/blob/main/Tezos_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://api.tzstats.com
#wallet: tz1PTFuRziyh2t9gak1mMiVu3zvTHAsPE92S

In [ ]:
import requests
import pandas as pd
import numpy as np


In [ ]:
ledger_1 = 'tz1i2tE6hic2ASe9Kvy85ar5hGSSc58bYejT'
corp_hotwallet = 'tz1isuiukJmWsp9DcuQLcSgPMqG3V8ZiErQT'
ledger_3 = 'tz1LWWJqCzKFA1BxGWXdH6ZEmqTktC1goCRx'
thanos_1 = 'tz1PTFuRziyh2t9gak1mMiVu3zvTHAsPE92S'
thanos_2 = 'tz1WWFozrxaTxc1FmswEzQ5adrJTcbw5E5CJ'
miniprod_1 = 'tz1fhrpoVnzc1jH9uUtAsk8toDLcFbeYV6ZJ'
aws_usds_issuer = 'tz1KokRHjd3n4BUSDSojeQ4HwttV6pW1gjeY'

tezos_wallets = [ledger_1, corp_hotwallet, ledger_3, thanos_1, thanos_2, miniprod_1, aws_usds_issuer]


In [ ]:
def get_transactions(address):
    """this is the Best function for pulling transactions currently"""

    transactions = []
    offset = 0
    while True:
        url = f"https://api.tzstats.com/explorer/account/{address}/operations?type=transaction&limit=100&offset={offset}"
        response = requests.get(url)
        if response.status_code != 200:
            break
        data = response.json()
        transactions += data
        offset += 100
        if len(data) < 100:
            break
    return pd.DataFrame(transactions)


In [ ]:
wallet = ledger_1
df = get_transactions(wallet)

In [ ]:
df

In [ ]:
df.columns

Index(['id', 'hash', 'type', 'block', 'time', 'height', 'cycle', 'counter',
       'op_n', 'op_p', 'status', 'is_success', 'is_contract', 'gas_limit',
       'gas_used', 'fee', 'parameters', 'code_hash', 'sender', 'receiver',
       'confirmations', 'volume', 'storage_limit', 'storage_paid', 'burned'],
      dtype='object')

In [ ]:
#grabs the entrypoint type
if 'parameters' not in df.columns:
   df['parameters'] = np.nan

df['entrypoint_type'] = df['parameters'].apply(lambda x: x['entrypoint'] if pd.notna(x) else None)

In [ ]:
# def extract_transfer_info(data):
#     try:
#         entrypoint = data.get('entrypoint')
#         if entrypoint != 'transfer':
#             return None, None, None
#         value = data.get('value')[0]
#         from_ = value.get('from_')
#         txs = value.get('txs')[0]
#         amount = txs.get('amount')
#         to_ = txs.get('to_')
#         return from_, amount, to_
#     except:
#         return None, None, None

# df = df.assign(transfer_from=df['parameters'].apply(lambda x: extract_transfer_info(x)[0]),
#                transfer_amount=df['parameters'].apply(lambda x: extract_transfer_info(x)[1]),
#                transfer_to=df['parameters'].apply(lambda x: extract_transfer_info(x)[2]))

# add in "if column existys"

def extract_transfer_info(data):
    try:
        entrypoint = data.get('entrypoint')
        if entrypoint != 'transfer':
            return None, None, None
        value = data.get('value')
        if type(value) == list:
            value = value[0]
            from_ = value.get('from_')
            txs = value.get('txs')[0]
            amount = txs.get('amount')
            to_ = txs.get('to_')
        else:
            from_ = value.get('from')
            to_ = value.get('to')
            amount = value.get('value')
        return from_, amount, to_
    except:
        return None, None, None


df = df.assign(transfer_from=df['parameters'].apply(lambda x: extract_transfer_info(x)[0]),
              transfer_amount=df['parameters'].apply(lambda x: extract_transfer_info(x)[1]),
              transfer_to=df['parameters'].apply(lambda x: extract_transfer_info(x)[2]))



In [ ]:
#unique_entrypoint_types = df['entrypoint_type'].unique()
#print(unique_entrypoint_types)

#thanos 1: ['configure_minter' 'transfer' None 'mint' 'burn']


In [ ]:
df

In [ ]:
df.iloc[:, -20:]

In [ ]:
#@title df reconstruction to make betwave compatable
df = df.loc[df['is_success'] == True]
#add XTZ to new column amountTicker if there's a value in amount
df.loc[df['volume'].notna(), 'amountTicker'] = 'XTZ'
#add XTZ to a new column titled feeTicker
df = df.assign(feeTicker = 'XTZ')

#if the transaction is a transfer, move the relevant amounts receivers
df = df.assign(volume_is_na = df['volume'].isna())

df.loc[df['volume_is_na'] & (df['entrypoint_type'] == 'transfer'), 'volume'] = df['transfer_amount']
df.loc[df['volume_is_na'] & (df['entrypoint_type'] == 'transfer'), 'amountTicker'] = df['receiver']

df.loc[df['entrypoint_type'] == 'transfer', 'sender'] = df['transfer_from']
df.loc[df['entrypoint_type'] == 'transfer', 'receiver'] = df['transfer_to']

#adjusting for update_operators
# delete rows where 'entrypoint_type' equals 'update_operators', 'fee' equals NaN and 'volume' equals NaN
df = df.drop(df.loc[(df['entrypoint_type'] == 'update_operators') & (df['fee'].isna()) & (df['volume'].isna())].index)
# update rows where 'entrypoint_type' equals 'update_operators', 'volume' equals NaN but 'fee' is not NaN
df.loc[(df['entrypoint_type'] == 'update_operators') & (df['volume'].isna()) & (df['fee'].notna()), 'volume'] = 0

#addusting for investLiquidity
# delete rows where 'entrypoint_type' equals 'tokenToTezPayment' and 'volume' and 'fee' are NaN
df = df.drop(df.loc[(df['entrypoint_type'] == 'tokenToTezPayment') & (df['volume'].isna()) & (df['fee'].isna())].index)

# update rows where 'entrypoint_type' equals 'tokenToTezPayment' and 'volume' or 'fee' is NaN
df.loc[(df['entrypoint_type'] == 'tokenToTezPayment') & (df['volume'].isna()), 'volume'] = 0
df.loc[(df['entrypoint_type'] == 'tokenToTezPayment') & (df['fee'].isna()), 'fee'] = 0


#adjusting for divestLiquidity
# delete rows where 'entrypoint_type' equals 'divesLiquidity' and 'volume' and 'fee' are NaN
df = df.drop(df.loc[(df['entrypoint_type'] == 'divesLiquidity') & (df['volume'].isna()) & (df['fee'].isna())].index)

# update rows where 'entrypoint_type' equals 'divesLiquidity' and 'volume' or 'fee' is NaN
df.loc[(df['entrypoint_type'] == 'divesLiquidity') & (df['volume'].isna()), 'volume'] = 0
df.loc[(df['entrypoint_type'] == 'divesLiquidity') & (df['fee'].isna()), 'fee'] = 0

#turn fee values to zero
df['fee'].fillna(0, inplace=True)


#various data cleaning
df.loc[df['amountTicker'] == 'KT1REEb5VxWRjcHm5GzDMwErMmNFftsE5Gpf', 'amountTicker'] = 'USDS'
df.loc[df['amountTicker'] =='KT1LN4LPSqTMS7Sd2CJw4bbDGRkMv2t68Fy9', 'amountTicker'] = 'USDT'

df['volume'] = pd.to_numeric(df['volume'])
df.loc[(df['volume_is_na'] == True) & (df['entrypoint_type'] == 'transfer'), 'volume'] = df['volume']/1000000

own_wallet = wallet
tezos_wallets.remove(own_wallet)

df.loc[(df['sender'] == own_wallet) & (df['burned'].notna()), 'fee'] += df['burned']
df.loc[df['sender'] != own_wallet, 'fee'] = 0

df.loc[:, 'transactionType'] = np.where(
      df.loc[:,'receiver'] == own_wallet, #if from wallet
      'deposit',                 #if condition is met
      'withdrawal'                     #if condition is not met
  )

accountid = 'test'#build into function later

df = df.assign(cost = '')
df = df.assign(costTicker = '')
df = df.assign(memo = '') #might turn into a f string variable
df = df.assign(accountId = f'{accountid}') #account id is wallet id from BitWave
df = df.assign(contactId = '')
df = df.assign(categoryId = '')
df = df.assign(remoteContactId = '')
#df = df.assign(taxExempt = 'FALSE')
df = df.assign(id = 'test')
df['rownum'] = np.arange(df.shape[0])
df['id'] = df['id'] + df['rownum'].astype(str)
df.drop('rownum', axis=1, inplace=True)



df['taxExempt'] = np.where((df['transactionType'] == 'withdrawal') & (df['receiver'].isin(tezos_wallets)), 'TRUE',
                            np.where((df['transactionType'] == 'deposit') & (df['sender'].isin(tezos_wallets)), 'TRUE', 'FALSE'))



df['time'] = pd.to_datetime(df['time'])

df.rename(columns = {'volume': 'amount', 'hash': 'blockchainId',}, inplace=True)


df = df[['id','remoteContactId','amount','amountTicker','cost','costTicker','fee','feeTicker','time','blockchainId','memo','transactionType','accountId','contactId','categoryId','taxExempt']]
df['amountTicker'].fillna('XTZ', inplace=True)

df = df.sort_values(by='time', ascending=True)

df['amount'] = df['amount'].fillna(0)

df

,id,remoteContactId,amount,amountTicker,cost,costTicker,fee,feeTicker,time,blockchainId,memo,transactionType,accountId,contactId,categoryId,taxExempt
20,test20,,293.232418,XTZ,,,0.000000,XTZ,2021-02-08 09:02:39+00:00,ooYHNrVfLc6ncFKMjMmWT3wrdsZHNy62NpRASm1CiRnLvz...,,deposit,test,,,FALSE
19,test19,,0.000000,XTZ,,,0.015463,XTZ,2021-02-09 05:03:51+00:00,opCg43NfGXUxrF9QpqAHZpNP5U5sDstQCnweYTnRkLDtym...,,withdrawal,test,,,FALSE
18,test18,,0.000000,XTZ,,,0.024778,XTZ,2021-02-10 01:59:59+00:00,ooFXRgiXvdMwiNd6o5u4wdcpsNiMfcXUs2M9cNrme7uNvz...,,withdrawal,test,,,FALSE
17,test17,,1.000000,XTZ,,,0.067527,XTZ,2021-02-10 02:07:39+00:00,ooUtakW5BWWNYnWKVx2ggoJzDWv3BktArJ5HZo7WZAyAfd...,,withdrawal,test,,,FALSE
16,test16,,5.000000,USDS,,,0.024222,XTZ,2021-02-10 02:09:39+00:00,oonM5b7eRccNxt8rCv7n91NtPrxSSUv4TF4XTHVx9wCk2h...,,withdrawal,test,,,FALSE
15,test15,,0.000000,XTZ,,,0.015479,XTZ,2021-02-10 02:23:39+00:00,onyjyQjYS8P4m5gY6rrHd3ziJ1uAU7i9uPf4L1JfMsZsD4...,,withdrawal,test,,,FALSE
14,test14,,0.000000,XTZ,,,0.006515,XTZ,2021-02-10 02:45:19+00:00,onvbkdo1LmgfYuMaudccyCHQ87AZmYFPkm5uJw38b9GShG...,,withdrawal,test,,,FALSE
13,test13,,0.000000,XTZ,,,0.006568,XTZ,2021-02-11 09:07:18+00:00,onixhm2D83c11hbJw4gLCC28axXU67sE8pF5URppC4WkPh...,,withdrawal,test,,,FALSE
12,test12,,1.000000,USDS,,,0.023934,XTZ,2021-02-15 04:55:32+00:00,opLP43KPejDpwRVbntXdyxHjCvX3g4LEh3oTZN443EMrCj...,,withdrawal,test,,,TRUE
11,test11,,0.000000,XTZ,,,0.008030,XTZ,2021-03-23 08:15:38+00:00,opEJkFG821eyQLwJbyYsNM4cJogirfQspPzAoLmApGb5Vw...,,withdrawal,test,,,FALSE


In [ ]:
def reconcile(df):
  """reconciles the dataframe"""

  df_tx = df[['amount', 'amountTicker', 'time', 'transactionType']]
  df_fee = df[['fee', 'feeTicker', 'time', 'transactionType']]

  df_fee.rename(columns={'fee': 'amount', 'feeTicker':'amountTicker'}, inplace=True)


  new_df = pd.concat([df_tx, df_fee], ignore_index=True)

  new_df['total'] = new_df['amount']
  new_df.loc[new_df['transactionType'] == 'withdrawal', 'total'] = - new_df['total']


  new_df = new_df.groupby('amountTicker')['total'].sum().reset_index()

  return new_df

In [ ]:
reconcile(df)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,amountTicker,total
0,USDS,-314.203436
1,USDT,-50.000000
2,XTZ,292.046482


In [ ]:
df.dtypes


id                              object
remoteContactId                 object
amount                         float64
amountTicker                    object
cost                            object
costTicker                      object
fee                            float64
feeTicker                       object
time               datetime64[ns, UTC]
blockchainId                    object
memo                            object
transactionType                 object
accountId                       object
contactId                       object
categoryId                      object
taxExempt                       object
dtype: object

In [ ]:


address = ledger_1

def get_transfers(address):
    url = f"https://api.tzkt.io/v1/accounts/{address}/operations"
    response = requests.get(url)
    transfers = response.json()
    df = pd.DataFrame(transfers)
    return df

In [ ]:
df_t = get_transfers('tz1isuiukJmWsp9DcuQLcSgPMqG3V8ZiErQT')

In [ ]:
df_t

,type,id,level,timestamp,block,hash,counter,sender,gasLimit,gasUsed,storageLimit,storageUsed,bakerFee,storageFee,allocationFee,target,amount,status,hasInternals
0,transaction,420519405944832,3012372,2022-12-29T23:31:29Z,BMFGMGtQfpFfM9a5tLnHa2Z1ZTJV2SzSVLFhyuPnJochZL...,oouvWVczNjETkBd7nHgaGC9Aos9qPCUCUFFUTki3qSYxTs...,12473282,{'address': 'tz1isuiukJmWsp9DcuQLcSgPMqG3V8ZiE...,10600,1001,257,0.0,1500,0.0,64250.0,{'address': 'tz1Wzcj4vr2gre2WeAg2mEXKqm7BnpvsP...,4.271246e+08,applied,False
1,transaction,368805071028224,2849879,2022-11-02T19:13:14Z,BL98NEtQnoRwEgGnBupUaDD52uHeyz4yxUQPnMf9BaMEA7...,oo6RRCRY64zKyb3rC7peZQwdYKZmDAdR6emTUjsAnUkW9K...,22904456,{'address': 'tz2QReH2vARQFQ6JiJ2ggwakdoW6TxBmF...,10600,1001,300,0.0,1420,0.0,0.0,{'address': 'tz1isuiukJmWsp9DcuQLcSgPMqG3V8ZiE...,9.500000e+06,applied,False
2,transaction,260907616698368,2495441,2022-07-01T01:13:59Z,BMc5an3BeofiMUSRyYxz5Mjw9gkjY359mBikaqgyZaaMFk...,opZzAXjaRmg5VEvTj5ewUxxwRYYjeeDsniMMSHYbh32fzw...,12473281,{'address': 'tz1isuiukJmWsp9DcuQLcSgPMqG3V8ZiE...,10300,1451,257,0.0,1500,0.0,64250.0,{'address': 'tz2T14rrG2zwHrQFNM7M7rqBRVT4zHDDW...,1.000000e+06,applied,False
3,transaction,71016735834112,1620815,2021-08-19T08:11:20Z,BLPBs7Mp1AaRMNPn4K2ACneqy3iu1jaMj5G8TC1JkPbTno...,oonSW2ktypCgwgEh969VtBN7edqcnbxNSxe5wopzkv8Rut...,2341230,"{'alias': 'Binance', 'address': 'tz1SiPXX4MYGN...",15400,1420,300,0.0,2940,0.0,0.0,{'address': 'tz1isuiukJmWsp9DcuQLcSgPMqG3V8ZiE...,2.099717e+07,applied,False
4,transaction,61816826757120,1548714,2021-07-08T20:57:50Z,BMZR8tM7hhQAiQFJVJFAvg7PXfnCftgJXVh5xEwtj6dPXu...,opLuoNcYHMW8sLYanyivCtPnEBCNk8gCyUXnJhj9Kd2U76...,12473280,{'address': 'tz1isuiukJmWsp9DcuQLcSgPMqG3V8ZiE...,10300,1427,257,0.0,1500,0.0,0.0,{'address': 'tz1ZchY2SfcP3GCLCBfGcCeTL2FyiWvL2...,2.600000e+10,applied,False
5,transaction,61702481641472,1548329,2021-07-08T14:24:42Z,BMTcJPnoVi3Gpmwc7rKoeJNVQNQWDJCB2ewWeQyiDiuRMN...,oodbYCPpy7c2AhBREyA1FvkiiogRXnCwSGk7ZEjjWK3nw8...,3954732,{'address': 'tz1UBTk5Jo18PAQcFWZhWS2fxt4S7Qbd5...,18000,1427,257,0.0,30000,0.0,0.0,{'address': 'tz1isuiukJmWsp9DcuQLcSgPMqG3V8ZiE...,2.609403e+10,applied,False
6,transaction,60236385746944,1535497,2021-06-28T23:23:22Z,BM4Tv9CBheXnAMZqJV5rFJDEYhoUetJRBdExZ6q2Ndtezf...,opJqpzzzQwXCjGi2Tz8KYn1L6zUmxha4uEN3Qb1uUbfE2G...,12473279,{'address': 'tz1isuiukJmWsp9DcuQLcSgPMqG3V8ZiE...,10300,1427,257,0.0,1500,0.0,64250.0,{'address': 'tz1TMAKuBEbie8R9M9K4q9A1pdxRSGW1S...,1.000000e+06,applied,False
7,transaction,55944053522432,1501324,2021-06-04T19:44:06Z,BLRqDBCoT6kUdLqejTxCvauLYQrpLn5cAdpm1HsTVbvKmJ...,oovKiLkRTnsdDE3DDMNdSBbj27CbhGf4pwMfLCFgpKk2tL...,12473278,{'address': 'tz1isuiukJmWsp9DcuQLcSgPMqG3V8ZiE...,10300,1427,257,0.0,1500,0.0,0.0,{'address': 'tz1ZchY2SfcP3GCLCBfGcCeTL2FyiWvL2...,8.500000e+09,applied,False
8,transaction,55564869566464,1498429,2021-06-02T18:31:30Z,BKkMMBqbFKgLs9qacSeSkXP57CJ7iePNZfbMMdsKvDmLNZ...,onsffu5DDEQ2RvxtGXfMWjK3bCgTLEd4w9WJUfJerkKCuN...,3954702,{'address': 'tz1UBTk5Jo18PAQcFWZhWS2fxt4S7Qbd5...,18000,1427,257,0.0,30000,0.0,0.0,{'address': 'tz1isuiukJmWsp9DcuQLcSgPMqG3V8ZiE...,7.910319e+09,applied,False
9,transaction,55496745680896,1497874,2021-06-02T08:59:02Z,BLq4TaF5pot7wrVLoxLkt7KUhREA5Ru6zVBrZnYnKhnDuv...,onnmgYSftR1KuypMaHPw7bgZZVQ1TDp9NASfK8Bdosf32v...,3954699,{'address': 'tz1UBTk5Jo18PAQcFWZhWS2fxt4S7Qbd5...,18000,1427,257,0.0,30000,0.0,0.0,{'address': 'tz1isuiukJmWsp9DcuQLcSgPMqG3V8ZiE...,1.000000e+06,applied,False
